In [1]:
from tools import get_puzzle, show_problem_1
TODAY = 22
puzzle = get_puzzle(TODAY)
show_problem_1(puzzle)


<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 21 - Advent of Code 2022</title>
<!--[if lt IE 9]><script src="/static/html5.js"></script><![endif]-->
<link href='//fonts.googleapis.com/css?family=Source+Code+Pro:300&subset=latin,latin-ext' rel='stylesheet' type='text/css'/>
<link rel="stylesheet" type="text/css" href="/static/style.css?30"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?0" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until yo

## --- Day 21: Monkey Math ---


The [monkeys](11) are back! You're worried they're going to try to steal your stuff again, but it seems like they're just holding their ground and making various monkey noises at you.


Eventually, one of the elephants realizes you don't speak monkey and comes over to interpret. As it turns out, they overheard you talking about trying to find the grove; they can show you a shortcut if you answer their **riddle** .


Each monkey is given a **job** : either to **yell a specific number** or to **yell the result of a math operation** . All of the number-yelling monkeys know their number from the start; however, the math operation monkeys need to wait for two other monkeys to yell a number, and those two other monkeys might **also** be waiting on other monkeys.


Your job is to **work out the number the monkey namedroot``will yell** before the monkeys figure it out themselves.


For example:


```
 root: pppw + sjmn
 dbpl: 5
 cczh: sllz + lgvd
 zczc: 2
 ptdq: humn - dvpt
 dvpt: 3
 lfqf: 4
 humn: 5
 ljgn: 2
 sjmn: drzm * dbpl
 sllz: 4
 pppw: cczh / lfqf
 lgvd: ljgn * ptdq
 drzm: hmdt - zczc
 hmdt: 32

```


Each line contains the name of a monkey, a colon, and then the job of that monkey:


+ A lone number means the monkey's job is simply to yell that number.
+ A job likeaaaa + bbbb``means the monkey waits for monkeysaaaa``andbbbb``to yell each of their numbers; the monkey then yells the sum of those two numbers.
+ aaaa - bbbb``means the monkey yellsaaaa``'s number minusbbbb``'s number.
+ Jobaaaa * bbbb``will yellaaaa``'s number multiplied bybbbb``'s number.
+ Jobaaaa / bbbb``will yellaaaa``'s number divided bybbbb``'s number.


So, in the above example, monkeydrzm``has to wait for monkeyshmdt``andzczc``to yell their numbers. Fortunately, bothhmdt``andzczc``have jobs that involve simply yelling a single number, so they do this immediately:32``and2``. Monkeydrzm``can then yell its number by finding32``minus2``:30``.


Then, monkeysjmn``has one of its numbers (30``, from monkeydrzm``), and already has its other number,5``, fromdbpl``. This allows it to yell its own number by finding30``multiplied by5``:150``.


This process continues untilroot``yells a number:152``.


However, your actual situation involvesconsiderably more monkeys. **What number will the monkey namedroot``yell?** 




In [4]:
import re
dbg = False

MOVE_DOWN = 'v'
MOVE_UP = '^'
MOVE_LEFT = '<'
MOVE_RIGHT = '>'

TRACES = [MOVE_RIGHT,MOVE_DOWN,MOVE_LEFT,MOVE_UP]
DIRECTIONS = [(0,1),(1,0),(0,-1),(-1,0)]
ROTATE_LEFT = 'L'
ROTATE_RIGHT = 'R'

SPACE = '.'
VOID = ' '
SOLID = '#'



class Map():

    def __init__(self, data):
        self.rows = [[x for x in row] for row in data[:-2]]
        self.width = max([len(row) for row in self.rows])
        self.height = len(self.rows)
        self.path = data[-1].strip()
        self.position = (0,self.initial_position())
        self.direction = 0 # the index in DIRECTIONS
        
        self.steps = self._iterator()
    
    def initial_position(self):
        for col, val in enumerate(self.rows[0]):
            if val == SPACE:
                if dbg: print(f"initial position (0,{col})")
                return col
    
    def __repr__(self):
        return "\n".join(["".join(line) for line in self.rows])

    
    def _iterator(self):
        matches = re.finditer(r"(\d+)([LR]?)", self.path)
        for match in matches:
            for group in match.groups():
                try:
                    group = int(group)
                except ValueError:
                    pass
                if group: # an empty char is coming in due to regex group 2
                    yield group

                    
                        
    def find_next(self):
        curr_row, curr_col = self.position
        next_thing = None
        while not next_thing:
            try:
                curr_row = curr_row + DIRECTIONS[self.direction%4][0]
                curr_col = curr_col + DIRECTIONS[self.direction%4][1] 
                
            except IndexError:
                if dbg: print (f"{curr_row}, {curr_col}")
                if dbg: print(self)
                raise IndexError
                
            if curr_col+1 > self.width:
                curr_col = 0
            if curr_col < 0:
                curr_col = self.width -1
            if curr_row+1 > self.height:
                curr_row = 0
            if curr_row < 0:
                curr_row = self.height -1                
            try:
                #print (f"{curr_row}, {curr_col}")
                next_thing = self.rows[curr_row][curr_col]
                if next_thing == VOID:
                    next_thing = None
            except IndexError:
                pass
            
        return (curr_row, curr_col), next_thing
        
    def attempt_move(self):
        curr_row, curr_col = self.position
        next_pos, next_thing = self.find_next()
            
            
        if dbg: print(f"attempting {next_pos}  {next_thing}")
        if next_thing == SPACE:
            self.rows[curr_row][curr_col] = TRACES[self.direction%4]
            self.position = next_pos
            return True
        if next_thing in [VOID, MOVE_RIGHT,MOVE_DOWN,MOVE_LEFT,MOVE_UP]:
            self.rows[curr_row][curr_col] = TRACES[self.direction%4]
            self.position = next_pos
            return True
        return False
        
    def move(self):
        try:
            while True:
                _next = next(self.steps)
                if isinstance(_next, int):
                    if dbg: print(f"\nadvancing {_next}")
                    for _ in range(_next):
                        is_possible= self.attempt_move()
                        if not is_possible:
                            break

                    pass
                else:
                    if dbg: print(f"\nwill do a {_next}")
                    if _next == ROTATE_RIGHT:
                        self.direction +=1
                    elif _next == ROTATE_LEFT:
                        self.direction -=1
                    else:
                        raise ValueError("wrong direction")
                if dbg: print (f"{self.position} looking {TRACES[self.direction%4]}")            
            
        except StopIteration:
            self.rows[self.position[0]][self.position[1]] = "*"
            return False
        
      
    
    def password(self):
        """The final password is the sum of 1000 times the row, 
        4 times the column, and the facing."""
        return 1000 * (self.position[0]+1) + 4 * (self.position[1]+1) + self.direction%4


    
def solution_1(data, moves=0):
    mapa = Map(data)
    if dbg:
        print(mapa)
        print("\n===============\n\n")
    if moves:
        for _ in range(moves):
            mapa.move()
    else:
        while mapa.move():
            pass

    if dbg:
        print(mapa)

    
    return mapa.password()

assert solution_1(puzzle.test) == 6032
assert solution_1(puzzle.data) == 50412

print( f"Solution 1:  The password is {solution_1(puzzle.data)}")

Solution 1:  The password is 50412


In [34]:
from tools import show_problem_2
show_problem_2(puzzle)

## --- Part Two ---


Due to some kind of monkey-elephant-human mistranslation, you seem to have misunderstood a few key details about the riddle.


First, you got the wrong job for the monkey namedroot``; specifically, you got the wrong math operation. The correct operation for monkeyroot``should be=``, which means that it still listens for two numbers (from the same two monkeys as before), but now checks that the two numbers **match** .


Second, you got the wrong monkey for the job starting withhumn:``. It isn't a monkey - it's **you** . Actually, you got the job wrong, too: you need to figure out **what number you need to yell** so thatroot``'s equality check passes. (The number that appears afterhumn:``in your input is now irrelevant.)


In the above example, the number you need to yell to passroot``'s equality test is301``. (This causesroot``to get the same number,150``, from both of its monkeys.)


 **What number do you yell to passroot``'s equality test?** 




In [5]:

SIZE = 50

class Map():

    def __init__(self, data, f_folding):
        self.rows = [[x for x in row] for row in data[:-2]]
        self.width = max([len(row) for row in self.rows])
        self.height = len(self.rows)
        self.path = data[-1].strip()
        self.position = (0,self.initial_position())
        self.direction = 0 # the index in DIRECTIONS
        self.folding = f_folding
        
        self.steps = self._iterator()
    
    def initial_position(self):
        for col, val in enumerate(self.rows[0]):
            if val == SPACE:
                if dbg: print(f"initial position (0,{col})")
                return col
    
    def __repr__(self):
        return "\n".join(["".join(line) for line in self.rows])

    
    def _iterator(self):
        matches = re.finditer(r"(\d+)([LR]?)", self.path)
        for match in matches:
            for group in match.groups():
                try:
                    group = int(group)
                except ValueError:
                    pass
                if group: # an empty char is coming in due to regex group 2
                    yield group

    def folding2(self, curr_row, curr_col, curr_dir):
  
        if TRACES[curr_dir%4] == MOVE_DOWN:
            if row == SIZE-1:
            elif row == 2*SIZE -1:
            elif row == self.height -1:

        elif TRACES[curr_dir%4] == MOVE_UP:
            if row == 0:
            elif row == SIZE:
            elif row == 2*SIZE:

            
        elif TRACES[curr_dir%4] == MOVE_RIGHT:
            if col == SIZE-1:
            elif col == 2*SIZE -1:
            elif col == self.height -1:
        
        elif TRACES[curr_dir%4] == MOVE_LEFT:
            if col == 0:
            elif col == SIZE:
            elif col == 2*SIZE:

    #########################################################################################################
    
  def find_next(self):
        curr_row, curr_col = self.position
        next_thing = None
        while not next_thing:
            try:
                curr_row = curr_row + DIRECTIONS[self.direction%4][0]
                curr_col = curr_col + DIRECTIONS[self.direction%4][1] 
                
            except IndexError:
                if dbg: print (f"{curr_row}, {curr_col}")
                if dbg: print(self)
                raise IndexError
                
            if curr_col+1 > self.width:
                curr_col = 0
            if curr_col < 0:
                curr_col = self.width -1
            if curr_row+1 > self.height:
                curr_row = 0
            if curr_row < 0:
                curr_row = self.height -1                
            try:
                #print (f"{curr_row}, {curr_col}")
                next_thing = self.rows[curr_row][curr_col]
                if next_thing == VOID:
                    next_thing = None
            except IndexError:
                pass
            
        return (curr_row, curr_col), next_thing
    
    
    def folding1(self, row, col, direction):
  
        if TRACES[direction%4] == MOVE_DOWN:
            if row == SIZE-1:
                DO AS BEFORE
            
            elif row == 2*SIZE -1:
                
                if col < SIZE:  # 3->5
                    direction +=2  # turn 180
                    row = 3*SIZE -1
                    col = 3*SIZE -1 -col
                
                elif col < 2*SIZE: #3->5
                    direction -=1  # turn L
                    row = 4*SIZE -1 -col
                    col = 2*SIZE
                else:               #4->5
                    DOASBEROFE
            
            elif row == self.height -1:
                if col < 3 * SIZE:  # 5->2
                    direction +=2 # turn 180
                    col = 3*SIZE - col
                    row = 2*SIZE - 1
                    
                elif col < 4 * SIZE:  # 6->2
                    direction -=1  # turn L
                    row = 5*SIZE - col
                    col = 0


        elif TRACES[direction%4] == MOVE_UP:
            if row == 0:
            elif row == SIZE:
            elif row == 2*SIZE:

            
        elif TRACES[direction%4] == MOVE_RIGHT:
            if col == SIZE-1:
            elif col == 2*SIZE -1:
            elif col == self.height -1:
        
        elif TRACES[direction%4] == MOVE_LEFT:
            if col == 0:
            elif col == SIZE:
            elif col == 2*SIZE:

                
    #########################################################################################################
                
    def find_next(self):
        curr_row, curr_col = self.position
        curr_dir = self.direction
        next_thing = None

        while not next_thing:
            
            curr_row, curr_col, curr_dir = self.folding1(curr_row, curr_col, curr_dir)
           
            
            curr_row = curr_row + DIRECTIONS[curr_dir%4][0]
            curr_col = curr_col + DIRECTIONS[curr_dir%4][1] 
        

            
            
            #print (f"{curr_row}, {curr_col}")
            next_thing = self.rows[curr_row][curr_col]
            
        return (curr_row, curr_col), next_thing
        
    def attempt_move(self):
        curr_row, curr_col = self.position
        next_pos, next_thing = self.find_next()
            
            
        if dbg: print(f"attempting {next_pos}  {next_thing}")
        if next_thing == SPACE:
            self.rows[curr_row][curr_col] = TRACES[self.direction%4]
            self.position = next_pos
            return True
        if next_thing in [VOID, MOVE_RIGHT,MOVE_DOWN,MOVE_LEFT,MOVE_UP]:
            self.rows[curr_row][curr_col] = TRACES[self.direction%4]
            self.position = next_pos
            return True
        return False
        
    def move(self):
        try:
            while True:
                _next = next(self.steps)
                if isinstance(_next, int):
                    if dbg: print(f"\nadvancing {_next}")
                    for _ in range(_next):
                        is_possible= self.attempt_move()
                        if not is_possible:
                            break

                    pass
                else:
                    if dbg: print(f"\nwill do a {_next}")
                    if _next == ROTATE_RIGHT:
                        self.direction +=1
                    elif _next == ROTATE_LEFT:
                        self.direction -=1
                    else:
                        raise ValueError("wrong direction")
                if dbg: print (f"{self.position} looking {TRACES[self.direction%4]}")            
            
        except StopIteration:
            self.rows[self.position[0]][self.position[1]] = "*"
            return False
        
      
    
    def password(self):
        """The final password is the sum of 1000 times the row, 
        4 times the column, and the facing."""
        return 1000 * (self.position[0]+1) + 4 * (self.position[1]+1) + self.direction%4


    
def solution_1(data, moves=0):
    mapa = Map(data)
    if dbg:
        print(mapa)
        print("\n===============\n\n")
    if moves:
        for _ in range(moves):
            mapa.move()
    else:
        while mapa.move():
            pass

    if dbg:
        print(mapa)

    
    return mapa.password()

assert solution_1(puzzle.test) == 6032
assert solution_1(puzzle.data) == 50412

print( f"Solution 1:  The password is {solution_1(puzzle.data)}")

Solution 1:  The password is 50412
